In [1]:
## path for mdules

import sys
#sys.path.insert(0,"/home/albert/lib/python")

import numpy as np
import xarray as xr
import glob

from netCDF4 import Dataset

sys.path.insert(0,"/home/henelle/Notebooks/git/xscale")
import xscale
import xscale.spectral.fft as xfft

import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import numpy.ma as ma

import matplotlib.cm as mplcm
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable

seq_cmap = mplcm.Blues
div_cmap = mplcm.seismic

import matplotlib.pyplot as plt
import matplotlib as mpl
from datetime import date

import seaborn as sns
sns.set(color_codes=True)

from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from math import cos, pi 

%matplotlib inline

In [2]:
## Dataset

dirfilt="/home/henelle/Notebooks/Data/"

In [3]:
## JFM & JAS files

tfileJFM = dirfilt + 'ACO_JFM_filtered.nc'
tfileJAS = dirfilt + 'ACO_JAS_filtered.nc'

In [4]:
params = {'font.weight':    'normal',
          'font.size':       int(14),
          'figure.titlesize': 'large',
          'legend.fontsize': int(12),
          'xtick.labelsize': int(14),
          'ytick.labelsize': int(14),
          'axes.labelsize':  int(14)}
mpl.rcParams.update(params)

In [5]:
def plot_surf(sub,data,lon,lat,vmin,vmax,cmap,title):
    
    ax = plt.subplot(sub,projection=ccrs.PlateCarree(central_longitude=0))
    ax.set_extent((-36, -26, 25, 40))
    land = cfeature.GSHHSFeature(scale='intermediate',levels=[1],facecolor=cfeature.COLORS['land'])
    
    ax.add_feature(land)
    gl = ax.gridlines(draw_labels=True, linestyle=':', color='black',
                          alpha=0.5)
    gl.xlabels_top = False
    gl.ylabels_right = False
    gl.xformatter = LONGITUDE_FORMATTER
    gl.yformatter = LATITUDE_FORMATTER
    ax.tick_params('both',labelsize=22)
    
    plt.pcolormesh(lon,lat,data,cmap=cmap,vmin=vmin,vmax=vmax )
    plt.colorbar(orientation='horizontal',pad=0.1,extend='both')
    plt.title(title,fontsize=18)
    #ax.text(0.57, -0.08, date, transform=ax.transAxes)

    ax.plot([-31, -28], [33, 33],color='black',linewidth=4)
    ax.plot([-31, -28], [36, 36],color='black',linewidth=4)
    ax.plot([-31, -31], [33, 36],color='black',linewidth=4)
    ax.plot([-28, -28], [33, 36],color='black',linewidth=4)

In [6]:
fJFM = xr.open_dataset(tfileJFM)
fJAS = xr.open_dataset(tfileJAS)

In [7]:
navlat = fJFM['lat']
navlon = fJFM['lon']

In [8]:
## RV variables

rvnotideJFM = fJFM['rv_notides_inst']
rv_fromSSH_ave24h_JFM = fJFM['rv_fromSSH_ave24h']

In [9]:
f = 1e-4

In [10]:
rv_notide_over_f_JFM = rvnotideJFM/f
rv_over_f_24h_JFM = rv_fromSSH_ave24h_JFM/f

In [11]:
Mrv_fromSSH_ave24h_JFM = np.mean(rv_over_f_24h_JFM,axis=0)
print(np.shape(Mrv_fromSSH_ave24h_JFM))

(397, 261)


fig = plt.figure(figsize=(14,7))

ax1 = fig.add_subplot(121)
ax2 = fig.add_subplot(122)


#ax1.pcolormesh(navlon,navlat,rv_notide_over_f_JFM[0,:,:],cmap=seq_cmap,vmin=-0.15,vmax=0.15)
#ax2.pcolormesh(navlon,navlat,rv_over_f_24h_JFM[0,:,:],cmap=seq_cmap,vmin=-0.15,vmax=0.15)

plot_surf(121,rv_notide_over_f_JFM[0,:,:],navlon,navlat,vmin=-0.15,vmax=0.15,cmap=seq_cmap,title='RV no tide')
plot_surf(122,rv_over_f_24h_JFM[0,:,:],navlon,navlat,vmin=-0.15,vmax=0.15,cmap=seq_cmap,title='24h average')


plt.suptitle('RV evolution during JFM',size=18,y=1.0)
ax1.set_title('RV no tide', size=14, y=1.0)
ax2.set_title('24h average', size=14, y=1.0)

#divider_ax1 = make_axes_locatable(ax1)
#divider_ax2 = make_axes_locatable(ax2)
#cax1 = divider_ax1.append_axes("bottom", size="5%", pad=0.05)
#cax2 = divider_ax2.append_axes("bottom", size="5%", pad=0.05)

#ax1.colorbar(orientation='horizontal',pad=0.1,extend='both',cax=cax1)
#ax2.colorbar(orientation='horizontal',pad=0.1,extend='both',cax=cax2)


def animate(i):
    
    ax1.pcolormesh(navlon,navlat,rv_notide_over_f_JFM[i,:,:],cmap=seq_cmap,vmin=-0.15,vmax=0.15)
    ax2.pcolormesh(navlon,navlat,rv_over_f_24h_JFM[i,:,:],cmap=seq_cmap,vmin=-0.15,vmax=0.15)
    
     
anim = FuncAnimation(fig, animate, frames=10, interval=100,repeat=False) # frames=100, interval=250
anim.save('RV_Acores_features.mp4',dpi=700)

import matplotlib.colors as colors

fig = plt.figure(figsize=(14,7))
ax1 = fig.add_subplot(121,projection=ccrs.PlateCarree())
ax2 = fig.add_subplot(122,projection=ccrs.PlateCarree())

plot_surf_anim(rv_notide_over_f_JFM[0,:,:],rv_over_f_24h_JFM[0,:,:],navlon,navlat,vmin=-0.15,vmax=0.15,cmap=seq_cmap,title='RV no tide')
plot_surf_anim(rv_notide_over_f_JFM[0,:,:],rv_over_f_24h_JFM[0,:,:],navlon,navlat,vmin=-0.15,vmax=0.15,cmap=seq_cmap,title='24h average')


#plt.suptitle('RV evolution during JFM',size=18,y=1.0)
#ax1.set_title('RV no tide', size=14, y=1.0)
#ax2.set_title('24h average', size=14, y=1.0)



def animate(i):
    
    pcolor1 = ax1.pcolormesh(navlon,navlat,rv_notide_over_f_JFM[i,:,:],cmap=seq_cmap,vmin=-0.15,vmax=0.15)
    pcolor2 = ax2.pcolormesh(navlon,navlat,rv_over_f_24h_JFM[i,:,:],cmap=seq_cmap,vmin=-0.15,vmax=0.15)
    
     
anim = FuncAnimation(fig, animate, frames=10, interval=100,repeat=False) # frames=100, interval=250
anim.save('RV_Acores_features.mp4',dpi=700)

In [12]:
def plot_surf_anim(data1,data2,lon,lat,vmin,vmax,cmap,title):
    
    fig = plt.figure(figsize=(17,17))
    ax1 = fig.add_subplot(121,projection=ccrs.PlateCarree())
    ax2 = fig.add_subplot(122,projection=ccrs.PlateCarree())
    
    norm_fld = colors.Normalize(vmin=vmin, vmax=vmax, clip=False)
    
    pcolor1 = plt.pcolormesh(lon,lat,data1[0,:,:],cmap=cmap,vmin=vmin,vmax=vmax,norm = norm_fld)
    pcolor2 = plt.pcolormesh(lon,lat,data2[0,:,:],cmap=cmap,vmin=vmin,vmax=vmax,norm = norm_fld)
    
    # Colorbar ax1
    cax1,kw1   = mpl.colorbar.make_axes(ax1,location='bottom',pad=0.05,shrink=0.7)
    out1       = fig.colorbar(pcolor1,cax=cax1,extend='max',**kw1)
    out1.set_label('$\zeta/f$')
    out1.ax.tick_params(labelsize=10)
    xticks_ax1 = np.linspace(vmin,vmax,num=5)
    out1.set_ticks(xticks_ax1)
    
    # Colorbar ax2
    cax2,kw2   = mpl.colorbar.make_axes(ax2,location='bottom',pad=0.05,shrink=0.7)
    out2       = fig.colorbar(pcolor2,cax=cax2,extend='max',**kw2)
    out2.set_label('$\zeta/f$')
    out2.ax.tick_params(labelsize=10)
    xticks_ax2 = np.linspace(vmin,vmax,num=5)
    out2.set_ticks(xticks_ax2)
    
    # Grid    
    gl1            = ax1.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,lw=1,color='gray',alpha=0.8, ls='--')
    gl1.xformatter = LONGITUDE_FORMATTER
    gl1.yformatter = LATITUDE_FORMATTER
    gl2            = ax2.gridlines(crs=ccrs.PlateCarree(),draw_labels=True,lw=1,color='gray',alpha=0.8, ls='--')
    gl2.xformatter = LONGITUDE_FORMATTER
    gl2.yformatter = LATITUDE_FORMATTER
    
    ax1.set_title('$\zeta/f$ no tide',size=14, y=1.1)
    ax2.set_title('$\zeta/f$ 24h average',size=14, y=1.1)
    plt.suptitle(title,size=18,y=0.8)

    # Area of interest
    ax1.plot([-31, -28], [33, 33],color='black',linewidth=4)
    ax1.plot([-31, -28], [36, 36],color='black',linewidth=4)
    ax1.plot([-31, -31], [33, 36],color='black',linewidth=4)
    ax1.plot([-28, -28], [33, 36],color='black',linewidth=4)
    
    ax2.plot([-31, -28], [33, 33],color='black',linewidth=4)
    ax2.plot([-31, -28], [36, 36],color='black',linewidth=4)
    ax2.plot([-31, -31], [33, 36],color='black',linewidth=4)
    ax2.plot([-28, -28], [33, 36],color='black',linewidth=4)
    
    
    
    def animate(i):

        pcolor1 = ax1.pcolormesh(lon,lat,data1[i,:,:],cmap=cmap,vmin=vmin,vmax=vmax)
        pcolor2 = ax2.pcolormesh(lon,lat,data2[i,:,:],cmap=cmap,vmin=vmin,vmax=vmax)


    anim = FuncAnimation(fig, animate, frames=100, interval=250,repeat=False,cache_frame_data=False) # frames=100, interval=250
    anim.save('RV_Acores_high.mp4',dpi=700) 
    #HTML(anim.to_html5_video())

In [ ]:
plot_surf_anim(rv_notide_over_f_JFM,rv_over_f_24h_JFM,navlon,navlat,vmin=-0.15,vmax=0.15,cmap=seq_cmap,title='$\zeta/f$ evolution during JFM')